<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/Mini_Projeto7_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mini-Projeto 7

## Sistema de Recomendação de Filmes da Netflix - Previsões

Este Jupyter Notebook é um bônus do Mini-Projeto 7. Aqui construímos um modelo de Machine Learning para nosso sistema de recomendação com o objetivo de prever a avaliação que o usuário dará a um filme. O modelo de Machine Learning será criado com o algoritmo XGBoost.

Execute este Jupyter Notebook depois de executar o Jupyter **01-DSA-Cap15-Mini-Projeto7.ipynb** disponível no Capítulo 15 do curso de Machine Learning da DSA.

Leia todos os comentários, inclua a função print() quando quiser compreender a saída de uma operação e estude todo o código usado neste trabalho.

Bons estudos.

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

## Instalando e Carregando os Pacotes

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

In [ ]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Imports
import os
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import sparse
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb
from datetime import datetime

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Preparação dos Dados

Vamos trabalhar com amostras dos dados, caso contrário o Jupyter Notebook vai levar muitas horas para ser executado.

A função abaixo será usada para extrair amostras de dados das matrizes esparsas criadas na parte 1 do Mini-Projeto.

In [ ]:
# Função para obter amostra da matriz esparsa
def gera_amostra_matriz_esparsa(sparse_matrix, num_users, num_movies, path, verbose = True):

    # Tupla: (row, col) e (rating) da matriz esparsa
    row_ind, col_ind, ratings = sparse.find(sparse_matrix)
    users = np.unique(row_ind)
    movies = np.unique(col_ind)

    # Random seed para reproduzir o processo aleatório
    np.random.seed(15)
    
    # Amostras de usuários e filmes
    sample_users = np.random.choice(users, num_users, replace = True)
    sample_movies = np.random.choice(movies, num_movies, replace = True)
    
    # Gera a máscara booleana
    mask = np.logical_and(np.isin(row_ind, sample_users), np.isin(col_ind, sample_movies))
    
    # Matriz esparsa com as amostras da matriz original
    amostra_matriz_esparsa = sparse.csr_matrix((ratings[mask], (row_ind[mask], col_ind[mask])), 
                                               shape = (max(sample_users) + 1, max(sample_movies) + 1))
    
    # Salva em disco
    print('Salvando em disco...')
    sparse.save_npz(path, amostra_matriz_esparsa)
    
    if verbose:
            print('Tarefa concluída.\n')
    
    return amostra_matriz_esparsa

### Gerando Amostra de Dados de Treino

In [ ]:
%%time

# Caminho onde está a matriz esparsa de treino gerada na Parte 1 do Mini-Projeto
caminho_matriz_treino_original = "dados/matriz_esparsa_treino.npz"

# Carregando a matriz esparsa
matriz_esparsa_treino_loaded = sparse.load_npz(caminho_matriz_treino_original)
print("Matriz Original Carregada.")

# Onde salvar a amostra
path = 'dados/amostra_matriz_esparsa_treino.npz'

# Obtemos avaliações de 1000 usuários a 100 filmes na matriz esparsa de treino
amostra_matriz_esparsa_treino = gera_amostra_matriz_esparsa(matriz_esparsa_treino_loaded, 
                                                            num_users = 1000, 
                                                            num_movies = 100, 
                                                            path = path)

### Gerando Amostra de Dados de Teste

In [ ]:
%%time

# Caminho onde está a matriz esparsa de treino gerada na Parte 1 do Mini-Projeto
caminho_matriz_teste_original = "dados/matriz_esparsa_teste.npz"

# Carregando a matriz de amostra, caso já exista
matriz_esparsa_teste_loaded = sparse.load_npz(caminho_matriz_teste_original)
print("Matriz Original Carregada.")

# Onde salvar a amostra
path = 'dados/amostra_matriz_esparsa_teste.npz'

# Obtemos avaliações de 200 usuários a 20 filmes na matriz esparsa de treino
amostra_matriz_esparsa_teste = gera_amostra_matriz_esparsa(matriz_esparsa_teste_loaded, 
                                                           num_users = 200, 
                                                           num_movies = 20,
                                                           path = path)

In [ ]:
# Resumo
print('Número de avaliações na matriz com amostras de treino: {}'.format(amostra_matriz_esparsa_treino.count_nonzero()))
print('Número de avaliações na matriz com amostras de teste: {}'.format(amostra_matriz_esparsa_teste.count_nonzero()))

Amostras criadas. Altere o número de usuários e número de filmes caso queira trabalhar com amostras maiores.

### Métricas Extraídas dos Dados

Vamos verificar algumas métricas a partir dos dados. Nosso modelo vai prever a avaliação do usuário ao filme.

In [ ]:
# Cria o dicionário
amostra_medias_treino = dict()

A função abaixo será usada para calcular a média de avaliações.

In [ ]:
def calcula_media_ratings(sparse_matrix, of_users):
    
    # Média de avaliações
    # 1 representa o eixo de usuários
    # 0 representa o eixo de filmes
    ax = 1 if of_users else 0 

    # Soma das avaliações
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    
    # Matriz booleana de avaliações (se um usuário avaliou ou não um filme)
    is_rated = sparse_matrix != 0
    
    # Número de avaliações de cada usuário ou filme
    no_of_ratings = is_rated.sum(axis = ax).A1
    
    # Ids da matriz esparsa, u de usuário e m de movie
    u,m = sparse_matrix.shape
    
    # Dicionário de usuários e suas avaliações
    average_ratings = {i:sum_of_ratings[i] / no_of_ratings[i] 
                       for i in range(u if of_users else m) 
                       if no_of_ratings[i] != 0}

    return average_ratings

Média global das avaliações dos filmes:

In [ ]:
# Média global
media_global = amostra_matriz_esparsa_treino.sum() / amostra_matriz_esparsa_treino.count_nonzero()
amostra_medias_treino['global'] = media_global
amostra_medias_treino

Média de avaliação por usuário:

In [ ]:
# Calcula a média de avaliação dos usuários
amostra_medias_treino['user'] = calcula_media_ratings(amostra_matriz_esparsa_treino, of_users = True)

In [ ]:
# Vamos extrair um dos usuários do dicionário de filmes (o objetivo aqui é apenas automatizar o processo)
um_usuario = [a for a, b in amostra_medias_treino['user'].items()][0]
um_usuario

In [ ]:
# Print
print('Média de Avaliação do Usuário ' + str(um_usuario) + ':', amostra_medias_treino['user'][um_usuario])

Média de avaliação por filme:

In [ ]:
# Calcula a média de avaliação dos filmes
amostra_medias_treino['movie'] =  calcula_media_ratings(amostra_matriz_esparsa_treino, of_users = False)

In [ ]:
# Vamos extrair um dos filmes do dicionário de filmes (o objetivo aqui é apenas automatizar o processo)
um_filme = [a for a, b in amostra_medias_treino['movie'].items()][0]
um_filme

In [ ]:
# Pring
print('Média de Avaliação do Filme ' + str(um_filme) + ':', amostra_medias_treino['movie'][um_filme])

## Formatando os Dados

Iremos construir um modelo de regressão, uma vez que desejamos prever as avaliações (valores numéricos). Vamos preparar os dados de treino e teste nas células abaixo.

Essas são as variáveis com as quais vamos construir o modelo:

Variáveis Preditoras (entrada):

- **GAvg** : Média global das avaliações


- **Avaliação de usuários semelhantes**:
    - sur1, sur2, sur3, sur4, sur5 (5 principais usuários similares a cada usuário que avaliou um filme)
    

- **Filmes semelhantes avaliados por um usuário**:
    - smr1, smr2, smr3, smr4, smr5 (5 principais filmes similares a cada filme avaliado)


- **UAvg** : Média de avaliações dos usuários


- **MAvg** : Média de avaliação do filme


Variável Alvo (saída):

- **rating** : Avaliação do filme dada por um usuário

### Preparando os Dados de Treino Para o Modelo de Regressão

In [ ]:
# Extraindo os dados da matriz de amostras
amostra_usuarios_treino, amostra_filmes_treino, amostra_avaliacoes_treino = sparse.find(amostra_matriz_esparsa_treino)

A célula abaixo leva bastante tempo para ser executada.

In [ ]:
%%time

# Verificamos se o arquivo já existe
if os.path.isfile('dados/dados_treino_reg.csv'):
    print("O arquivo já existe e não precisamos criar movamente..." )
else:
    print('Preparando {} tuplas para o dataset..\n'.format(len(amostra_medias_treino)))
    with open('dados/dados_treino_reg.csv', mode = 'w') as reg_data_file:
        count = 0
        for (user, movie, rating) in zip(amostra_usuarios_treino, amostra_filmes_treino, amostra_avaliacoes_treino):
             
            ###### Avaliação de um "filme" por usuários similares ao usuário corrente ######
            
            # Calcula usuário similar ao usuário corrente        
            user_sim = cosine_similarity(amostra_matriz_esparsa_treino[user], 
                                         amostra_matriz_esparsa_treino).ravel()
            
            # Obtém top users
            top_sim_users = user_sim.argsort()[::-1][1:] 
            
            # Obtém avaliações de usuários similares 
            top_ratings = amostra_matriz_esparsa_treino[top_sim_users, movie].toarray().ravel()
            
            # Top usuários similares até 5
            top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_users_ratings.extend([amostra_medias_treino['movie'][movie]]*(5 - len(top_sim_users_ratings))) 

            ##### Avaliações por usuário para filmes similares ao filme corrente #####
            
            # Calcula filmes similares ao filme corrente       
            movie_sim = cosine_similarity(amostra_matriz_esparsa_treino[:,movie].T, 
                                          amostra_matriz_esparsa_treino.T).ravel()
            
            # Top filmes
            top_sim_movies = movie_sim.argsort()[::-1][1:] 
            
            # Obtém avaliações do filme mais similar para o usuário corrente
            top_ratings = amostra_matriz_esparsa_treino[user, top_sim_movies].toarray().ravel()
            
            # Top usuários similares até 5
            top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movies_ratings.extend([amostra_medias_treino['user'][user]] * (5-len(top_sim_movies_ratings))) 

            ##### Prepara a linha que será armazenada no arquivo #####
            row = list()
            row.append(user)
            row.append(movie)
            
            # Adicionamos outros atributos 
            row.append(amostra_medias_treino['global']) 
            row.extend(top_sim_users_ratings)
            row.extend(top_sim_movies_ratings)
            row.append(amostra_medias_treino['user'][user])
            row.append(amostra_medias_treino['movie'][movie])

            row.append(rating)
            count = count + 1
            
            #if count == 10:
            #    break

            reg_data_file.write(','.join(map(str, row)))
            reg_data_file.write('\n')        
            if (count)%10000 == 0:
                print("Concluído para {} linhas----- {}".format(count, datetime.now() - start))

Carregamos o arquivo e colocamos em um dataframe.

In [ ]:
df_dados_treino_reg = pd.read_csv('dados/dados_treino_reg.csv', 
                               names = ['user', 
                                        'movie', 
                                        'GAvg', 
                                        'sur1', 
                                        'sur2', 
                                        'sur3', 
                                        'sur4', 
                                        'sur5',
                                        'smr1', 
                                        'smr2', 
                                        'smr3', 
                                        'smr4', 
                                        'smr5', 
                                        'UAvg', 
                                        'MAvg', 
                                        'rating'], 
                               header = None)

In [ ]:
# Dados
df_dados_treino_reg.head()

### Preparando os Dados de Teste Para o Modelo de Regressão

O processo aqui é igual ao que fizemos com dados de treino.

In [ ]:
# Extraindo os dados da matriz de amostras
amostra_usuarios_teste, amostra_filmes_teste, amostra_avaliacoes_teste = sparse.find(amostra_matriz_esparsa_teste)

In [ ]:
%%time

if os.path.isfile('dados/dados_teste_reg.csv'):
    print("O arquivo já existe e não precisamos criar movamente...")
else:

    print('Preparando {} tuplas para o dataset..\n'.format(len(amostra_avaliacoes_teste)))
    with open('dados/dados_teste_reg.csv', mode='w') as reg_data_file:
        count = 0 
        for (user, movie, rating)  in zip(amostra_usuarios_teste, amostra_filmes_teste, amostra_avaliacoes_teste):
            st = datetime.now()

            # Similaridade de usuários
            try:
                user_sim = cosine_similarity(amostra_matriz_esparsa_treino[user], 
                                             amostra_matriz_esparsa_treino).ravel()
                
                top_sim_users = user_sim.argsort()[::-1][1:] 
                top_ratings = amostra_matriz_esparsa_treino[top_sim_users, movie].toarray().ravel()
                top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
                top_sim_users_ratings.extend([amostra_medias_treino['movie'][movie]]*(5 - len(top_sim_users_ratings)))

            except (IndexError, KeyError):
                top_sim_users_ratings.extend([amostra_medias_treino['global']]*(5 - len(top_sim_users_ratings)))
            except:
                print(user, movie)
                raise

            # Similaridade de filmes
            try:
                movie_sim = cosine_similarity(amostra_matriz_esparsa_treino[:,movie].T, 
                                              amostra_matriz_esparsa_treino.T).ravel()
                
                top_sim_movies = movie_sim.argsort()[::-1][1:] 
                top_ratings = amostra_matriz_esparsa_treino[user, top_sim_movies].toarray().ravel()
                top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
                top_sim_movies_ratings.extend([amostra_medias_treino['user'][user]]*(5-len(top_sim_movies_ratings))) 
            except (IndexError, KeyError):
                top_sim_movies_ratings.extend([amostra_medias_treino['global']]*(5-len(top_sim_movies_ratings)))
            except :
                raise

            # Prepara os dados para gravar no arquivo
            row = list()
            row.append(user)
            row.append(movie)
            row.append(amostra_medias_treino['global']) 
            row.extend(top_sim_users_ratings)
            row.extend(top_sim_movies_ratings)

            try:
                row.append(amostra_medias_treino['user'][user])
            except KeyError:
                row.append(amostra_medias_treino['global'])
            except:
                raise

            try:
                row.append(amostra_medias_treino['movie'][movie])
            except KeyError:
                row.append(amostra_medias_treino['global'])
            except:
                raise

            row.append(rating)
            
            count = count + 1
            
            #if count == 5:
            #    break
    
            reg_data_file.write(','.join(map(str, row)))
            reg_data_file.write('\n')        
            if (count)%1000 == 0:
                print("Concluído em {} linhas----- {}".format(count, datetime.now() - start))

Carregamos o arquivo e colocamos em um dataframe.

In [ ]:
# Gera o dataset de teste
df_dados_teste_reg = pd.read_csv('dados/dados_teste_reg.csv', names = ['user', 
                                                                       'movie', 
                                                                       'GAvg', 
                                                                       'sur1', 
                                                                       'sur2', 
                                                                       'sur3', 
                                                                       'sur4', 
                                                                       'sur5',
                                                                       'smr1', 
                                                                       'smr2', 
                                                                       'smr3', 
                                                                       'smr4', 
                                                                       'smr5',
                                                                       'UAvg', 
                                                                       'MAvg', 
                                                                       'rating'], 
                                 header = None)

In [ ]:
df_dados_teste_reg.head()

## Construindo o Modelo de Machine Learning

A última etapa do trabalho é construir, treinar e avaliar o modelo.

In [ ]:
# Dicionários para avaliação do modelo
models_evaluation_train = dict()
models_evaluation_test = dict()

Abaixo algumas funções para executar o modelo.

In [ ]:
# Função para o cálculo do erro do modelo
def calcula_metricas(y_true, y_pred):
    rmse = np.sqrt(np.mean([ (y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ]))
    mape = np.mean(np.abs( (y_true - y_pred)/y_true )) * 100
    return rmse, mape

In [ ]:
# Função para treino e teste do modelo
def executa_modelo_xgboost(modelo, x_train, y_train, x_test, y_test, verbose = True):

    # Dicionários
    train_results = dict()
    test_results = dict()
    
    # Treinamento do modelo
    print('Treinando o modelo..')
    start = datetime.now()
    modelo.fit(x_train, y_train, eval_metric = 'rmse')
    print('Concluído. Tempo total: {}\n'.format(datetime.now() - start))

    # Calculando o erro do modelo nos dados de treino
    print('Calculando as Métricas com Dados de Treino.')
    start = datetime.now()
    y_train_pred = modelo.predict(x_train)
    rmse_train, mape_train = calcula_metricas(y_train.values, y_train_pred)
    
    # Grava os resultados
    train_results = {'rmse': rmse_train, 'mape' : mape_train, 'previsoes' : y_train_pred}
    
    if verbose:
        print('\nErro do Modelo em Dados de Treino')
        print('-'*30)
        print('RMSE : ', rmse_train)
        print('MAPE : ', mape_train)
        
    # Avaliando o modelo com dados de teste
    print('\nAvaliando o modelo com dados de teste.')
    y_test_pred = modelo.predict(x_test) 
    rmse_test, mape_test = calcula_metricas(y_true = y_test.values, y_pred = y_test_pred)
    
    # Grava os resultados
    test_results = {'rmse': rmse_test, 'mape' : mape_test, 'previsoes':y_test_pred}
    
    if verbose:
        print('\nErro do Modelo em Dados de Teste')
        print('-'*30)
        print('RMSE : ', rmse_test)
        print('MAPE : ', mape_test)
        
    return train_results, test_results

In [ ]:
# Seed
my_seed = 15
random.seed(my_seed)
np.random.seed(my_seed)

## Treinamento do Modelo

In [ ]:
# Prepara os dados de treino
x_treino = df_dados_treino_reg.drop(['user', 'movie', 'rating'], axis = 1)
y_treino = df_dados_treino_reg['rating']

In [ ]:
# Prepara os dados de teste
x_teste = df_dados_teste_reg.drop(['user', 'movie', 'rating'], axis = 1)
y_teste = df_dados_teste_reg['rating']

In [ ]:
# Cria o modelo de regressão com 100 estimadores
modelo_xgb = xgb.XGBRegressor(silent = False, random_state = 15, n_estimators = 100)

In [ ]:
# Treinamento do modelo
train_results, test_results = executa_modelo_xgboost(modelo_xgb, x_treino, y_treino, x_teste, y_teste)

In [ ]:
# Armazena os resultados da avaliação do modelo
models_evaluation_train['modelo_xgb'] = train_results
models_evaluation_test['modelo_xgb'] = test_results

In [ ]:
# Variáveis mais importantes para o modelo
xgb.plot_importance(modelo_xgb)
plt.show()

Além de construir o modelo também identificamos as variáveis mais relevantes. Observe que não há surpresa. As avaliações de usuários são determinantes para recomendar os filmes avaliados para outros usuários.

## Salvando o Resultado

In [ ]:
# Salva os resultados em disco
pd.DataFrame(models_evaluation_test).to_csv('dados/resultado.csv')
models = pd.read_csv('dados/resultado.csv', index_col = 0)
models.loc['rmse'].sort_values()

# Fim